In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

with open("my_knowledge.txt") as f:
    knowledge_text = f.read()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=20,
    length_function=len
)

chunks = text_splitter.split_text(knowledge_text)

print(f"We have {len(chunks)} chunks:")
for i, chunk in enumerate(chunks):
    print(f"--- Chunk {i+1} ---\n{chunk}\n")


We have 4 chunks:
--- Chunk 1 ---
Company Policy Manual:

--- Chunk 2 ---
- WFH Policy: All employees are eligible for a hybrid WFH schedule. Employees must be in the office on Tuesdays, Wednesdays, and Thursdays. Mondays

--- Chunk 3 ---
Thursdays. Mondays and Fridays are optional remote days.

--- Chunk 4 ---
- PTO Policy: Full-time employees receive 20 days of Paid Time Off (PTO) per year. PTO accrues monthly.



In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

with open("my_knowledge.txt") as f:
    knowledge_text = f.read()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=20
)

chunks = text_splitter.split_text(knowledge_text)

print(f"Chunks created: {len(chunks)}")


Chunks created: 4


In [7]:
for i, chunk in enumerate(chunks):
    print(f"--- Chunk {i+1} ---\n{chunk}\n")

--- Chunk 1 ---
Company Policy Manual:

--- Chunk 2 ---
- WFH Policy: All employees are eligible for a hybrid WFH schedule. Employees must be in the office on Tuesdays, Wednesdays, and Thursdays. Mondays

--- Chunk 3 ---
Thursdays. Mondays and Fridays are optional remote days.

--- Chunk 4 ---
- PTO Policy: Full-time employees receive 20 days of Paid Time Off (PTO) per year. PTO accrues monthly.



In [8]:
from sentence_transformers import SentenceTransformer

# 1. Load the embedding model
# 'all-MiniLM-L6-v2' is a fantastic, fast, and small model.
# It runs 100% on your local machine.
model = SentenceTransformer('all-MiniLM-L6-v2')

# 2. Embed all our chunks
# This will take a moment as it "reads" and "understands" each chunk.
chunk_embeddings = model.encode(chunks)

print(f"Shape of our embeddings: {chunk_embeddings.shape}")

Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Shape of our embeddings: (4, 384)


In [10]:
from sentence_transformers import SentenceTransformer

# 1. Load the embedding model
# 'all-MiniLM-L6-v2' is a fantastic, fast, and small model.
# It runs 100% on your local machine.
model = SentenceTransformer('all-MiniLM-L6-v2')

# 2. Embed all our chunks
# This will take a moment as it "reads" and "understands" each chunk.
chunk_embeddings = model.encode(chunks)

print(f"Shape of our embeddings: {chunk_embeddings.shape}")

Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Shape of our embeddings: (4, 384)


In [11]:
import faiss
import numpy as np

# Get the dimension of our vectors (e.g., 384)
d = chunk_embeddings.shape[1]

# 1. Create a FAISS index
# IndexFlatL2 is the simplest, most basic index. It calculates
# the exact distance (L2 distance) between our query and all vectors.
index = faiss.IndexFlatL2(d)

# 2. Add our chunk embeddings to the index
# We must convert to float32 for FAISS
index.add(np.array(chunk_embeddings).astype('float32'))

print(f"FAISS index created with {index.ntotal} vectors.")

FAISS index created with 4 vectors.


In [29]:
from transformers import pipeline

generator = pipeline(
    "question-answering",
    model="distilbert-base-uncased-distilled-squad"
)


config.json:   0%|          | 0.00/451 [00:00<?, ?B/s]

C:\Users\anjal\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\anjal\.cache\huggingface\hub\models--distilbert-base-uncased-distilled-squad. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/102 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [36]:
def answer_question(query):
    # 1. RETRIEVE
    query_embedding = model.encode([query]).astype("float32")
    distances, indices = index.search(query_embedding, k=2)

    # Distance threshold (tuned for MiniLM)
    if distances[0][0] > 1.0:
        return "I don't have that information."

    retrieved_chunks = [chunks[i] for i in indices[0]]
    context = " ".join(retrieved_chunks)

    # 2. GENERATE (QA-style)
    result = generator(
        question=query,
        context=context
    )

    # 3. QA CONFIDENCE CHECK
    if result["score"] < 0.3:
        return "I don't have that information."

    return result["answer"]


In [37]:
answer_question("What is the WFH policy?")


'All employees are eligible for a hybrid WFH schedule'

In [38]:
answer_question("What is the dental policy?")


"I don't have that information."